# CIFAR Table 1

TJ Kim

1.17.22

#### Summary:
- Make a table for Benign transferability and inter-boundary distance for following models
- Local benign, fedavg benign, fedEM benign

In [3]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


In [4]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

#### Local Benign

In [34]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedAvg"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 1
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 265.56it/s]


===> Initializing clients..


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:09<00:00,  8.66it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.297 | Train Acc: 10.823% |Test Loss: 2.298 | Test Acc: 10.893% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


Generate Dataset to be used throughout all analysis.

In [6]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

In [7]:
# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

Load local model.

In [35]:
setting = 'FedAvg'
num_models = 8

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

    # args_.save_path = 'weights/final/cifar/fig1/Local_Benign/'
    args_.save_path ='weights/final/cifar/fig3/local_defend'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
    # weights = np.load("weights/final/cifar/fig1/Local_Benign/train_client_weights.npy")
    weights = np.load('weights/final/cifar/fig3/local_defend/train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
#     args_.save_path = 'weights/final/cifar/fig1/FedAvg_Benign/'
    args_.save_path = 'weights/final/cifar/fig3/fed_avg_defend'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    # weights = np.load("weights/final/cifar/fig1/FedAvg_Benign/train_client_weights.npy")
    weights = np.load('weights/final/cifar/fig3/fed_avg_defend/train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
    # args_.save_path = 'weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/'
    args_.save_path = 'weights/final/cifar/fig3/fedem_defend/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/train_client_weights.npy")
    weights = np.load("weights/final/cifar/fig3/fedem_defend/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

Set Up Transfer Attack Scenario

In [36]:
logs_adv = []

for i in range(num_models):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities_target'] = None
    adv_dict['adv_similarities_untarget'] = None
    adv_dict['adv_target'] = None
    adv_dict['adv_miss'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['ib_distance_legit'] = None
    adv_dict['ib_distance_adv'] = None

    logs_adv += [adv_dict]


In [37]:
# Run Measurements for both targetted and untargeted analysis
victim_idxs = range(num_models)

t1 = Transferer(models_list=models_test, dataloader=dataloader)
t1.generate_victims(victim_idxs)

for adv_idx in victim_idxs:
    print("\t Adv idx:", adv_idx)
    # Perform Attacks
    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=500, iteration = 20,
                   target = 3, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.05, step_norm = "inf", eps = 4.5, eps_norm = 2)

    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)

    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
    logs_adv[adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
    logs_adv[adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
    logs_adv[adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

    # Miss attack
    t1.atk_params.set_params(batch_size=500, iteration = 20,
                   target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.05, step_norm = "inf", eps = 4.5, eps_norm = 2)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)
    logs_adv[adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)

	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7


Print Relevant Information

In [38]:
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
orig_sim = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_target = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_untarget = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_target = np.zeros([len(victim_idxs),len(victim_idxs)])
adv_miss = np.zeros([len(victim_idxs),len(victim_idxs)]) 

for adv_idx in range(len(victim_idxs)):
    for victim in range(len(victim_idxs)):
        orig_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
        orig_sim[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
        adv_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
        adv_sim_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
        adv_sim_untarget[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[4]][victim_idxs[victim]].data.tolist()
        adv_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[5]][victim_idxs[victim]].data.tolist()
        adv_miss[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[6]][victim_idxs[victim]].data.tolist()

In [42]:
avg_nondiag(adv_target)

0.21798112988471985

Set Up Interboundary Measure Scenario

In [31]:
num_trials = 200
batch_size = 5000
adv_idx = [0]

t1 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)
t1.base_nn_idx = 0
t1.victim_idx = [1,2,3,4]


dists_measure_legit = np.zeros([num_trials, len(t1.victim_idx)])
dists_measure_adv = np.zeros([num_trials, len(t1.victim_idx)])
dists_measure_adv_ensemble = np.zeros([num_trials, len(t1.victim_idx)])

for i in range(num_trials):
    print("num_trial:", i)
    t1 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)
    t1.base_nn_idx = 0
    t1.victim_idx = [1,2,3,4]

    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=500, iteration = 30,
                   target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.05, step_norm = "inf", eps = 3, eps_norm = 2)
    t1.set_adv_NN(t1.base_nn_idx)

    base_ep_legit, victim_eps_legit = t1.legitimate_direction(batch_size=batch_size, ep_granularity = 0.5, 
                                                              rep_padding = 1000, new_point = True,print_res = False)
    
    base_ep_adv, victim_eps_adv = t1.adversarial_direction(ep_granularity = 0.5, 
                                                              rep_padding = 1000, new_point = False,print_res = False)
    
    idx = 0
    for key, value in victim_eps_legit.items():
        dists_measure_legit[i,idx] = np.abs(base_ep_legit-value)
        idx+=1
        
    idx = 0
    for key, value in victim_eps_adv.items():
        dists_measure_adv[i,idx] = np.abs(base_ep_adv - value)
        idx+=1
    

num_trial: 0
num_trial: 1
num_trial: 2
num_trial: 3
num_trial: 4
num_trial: 5
num_trial: 6
num_trial: 7
num_trial: 8
num_trial: 9
num_trial: 10
num_trial: 11
num_trial: 12
num_trial: 13
num_trial: 14
num_trial: 15
num_trial: 16
num_trial: 17
num_trial: 18
num_trial: 19
num_trial: 20
num_trial: 21
num_trial: 22
num_trial: 23
num_trial: 24
num_trial: 25
num_trial: 26
num_trial: 27
num_trial: 28
num_trial: 29
num_trial: 30
num_trial: 31
num_trial: 32
num_trial: 33
num_trial: 34
num_trial: 35
num_trial: 36
num_trial: 37
num_trial: 38
num_trial: 39
num_trial: 40
num_trial: 41
num_trial: 42
num_trial: 43
num_trial: 44
num_trial: 45
num_trial: 46
num_trial: 47
num_trial: 48
num_trial: 49
num_trial: 50
num_trial: 51
num_trial: 52
num_trial: 53
num_trial: 54
num_trial: 55
num_trial: 56
num_trial: 57
num_trial: 58
num_trial: 59
num_trial: 60
num_trial: 61
num_trial: 62
num_trial: 63
num_trial: 64
num_trial: 65
num_trial: 66
num_trial: 67
num_trial: 68
num_trial: 69
num_trial: 70
num_trial: 71
nu

In [32]:
np.mean(np.average(dists_measure_legit,axis=0))

50.214375000000004

In [33]:
np.mean(np.average(dists_measure_adv,axis=0))

87.08125

#### Logs

In [ ]:
# Local - Benign
local_adv_miss = 0.37
local_adv_target = 0.07
local_orig_acc = 0.41
local_ibdist_legit = 39.52875
local_ibdist_adv = 49.74

# Fedavg - Benign
fedavg_adv_miss = 0.0
fedavg_adv_target = 0.86 
fedavg_orig_acc = 0.96
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - Benign
fedem_adv_miss = 0.12
fedem_adv_target = 0.43
fedem_orig_acc = 0.79
fedem_ibdist_legit = 9.265
fedem_ibdist_adv = 10.03

# Local - adv
local_adv_miss = 0.30
local_adv_target = 0.05
local_orig_acc = 0.30
local_ibdist_legit = 50.21
local_ibdist_adv = 87.08

# Fedavg - adv
fedavg_adv_miss = 0.352
fedavg_adv_target =  0.21
fedavg_orig_acc = 0.93
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - adv
fedem_adv_miss = 0.54
fedem_adv_target = 0.073
fedem_orig_acc = 0.71
fedem_ibdist_legit = 14.29
fedem_ibdist_adv = 48.97